In [2]:
import ndjson
import numpy as np
from datetime import datetime
import pandas as pd

# Importing Data

In [3]:
with open('./mimic-fhir/Patient.ndjson') as file:
    data = ndjson.load(file)
with open('./mimic-fhir/Condition.ndjson') as file:
    condition_data = ndjson.load(file)
with open('./mimic-fhir/Encounter.ndjson') as file:
    encounter_data = ndjson.load(file)
with open('./mimic-fhir/EncounterICU.ndjson') as file:
    encounter_icu_data = ndjson.load(file)
    

In [4]:
patient_id =[] #creating list of patient_id
for i in range(len(data)):
    patient_id.append(data[i]['id'])


In [5]:
patient_id[:5]

['0a8eebfd-a352-522e-89f0-1d4a13abdebc',
 '0c2243d2-987b-5cbd-8eb1-170a80647693',
 '13df78e7-150e-5eb7-be5f-5f62b2baee87',
 '158f3a39-e3d7-5e7a-93aa-57af894aadd9',
 '1ab119a5-aac8-5002-9d2f-b8ff69623387']

# Creating Dictionary 

## This is the format of the dictionay
<code>
{
    "patient_id_1": [
        {
            "condition_json": {...},
            "estimated_time": "2022-01-15T09:30:00Z"
        },
        {
            "condition_json": {...},
            "estimated_time": "2022-03-20T14:45:00Z"
        },
        ...
    ],
    "patient_id_2": [
        {
            "condition_json": {...},
            "estimated_time": "2021-08-02T11:15:00Z"
        },
        ...
    ],
    ...
}
</code>

    ],
    ...
}
...
}
\end{verbatim}

In [6]:
dict ={} #Dictionary which stores patient_id as key and other parameters as value
for id in patient_id:
    list = [] # To store value of specific patient_id
    for cond in condition_data:
        cond_dict={} #To store each condition and its estimated time
        if (id == cond['subject']['reference'][8:]): 
            cond_dict['condition_json'] = cond
            i_d = (cond['encounter']['reference'][10:])
            for encou in encounter_data:
                if i_d == encou['id']:
                    cond_dict['estimated_time'] = (encou['period']['start'])
            list.append(cond_dict)
    arr = np.array(list)
    dict[id] = arr
            
    
            

# Unix Format

In [7]:
def unix_format(time_string):
    # Parse the time string
    dt = datetime.fromisoformat(time_string.replace('Z', '+00:00'))
    # Convert to Unix timestamp
    unix_time = dt.timestamp()
    return unix_time


In [8]:
unix_format(dict[patient_id[0]][0]['estimated_time'])

6644651700.0

# Creating DataFrame

In [9]:
Column_name = ['pid',
               'time',
               'code',
               'description']
df = pd.DataFrame(columns=Column_name)
df

,pid,time,code,description


In [10]:
for p_id in dict:
    for val in dict[p_id]:
        code = (val['condition_json']['code']['coding'][0]['code'])
        display = (val['condition_json']['code']['coding'][0]['display'])
        time_unix = unix_format(val['estimated_time'])
        temp_duct ={
            'pid' : [p_id],
            'time' : [time_unix],
            'code' : [code],
            'description' : [display]
        }
        df2 = pd.DataFrame(temp_duct)
        df = pd.concat([df,df2])
        

C:\Users\himan\AppData\Local\Temp\ipykernel_32336\3877504377.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df2])


In [11]:
df

,pid,time,code,description
0,0a8eebfd-a352-522e-89f0-1d4a13abdebc,6.644652e+09,V462,"Other dependence on machines, supplemental oxygen"
0,0a8eebfd-a352-522e-89f0-1d4a13abdebc,6.642340e+09,78959,Other ascites
0,0a8eebfd-a352-522e-89f0-1d4a13abdebc,6.642340e+09,07071,Unspecified viral hepatitis C with hepatic coma
0,0a8eebfd-a352-522e-89f0-1d4a13abdebc,6.642340e+09,5715,Cirrhosis of liver without mention of alcohol
0,0a8eebfd-a352-522e-89f0-1d4a13abdebc,6.644652e+09,29680,"Bipolar disorder, unspecified"
...,...,...,...,...
0,fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,6.563037e+09,2724,Other and unspecified hyperlipidemia
0,fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,6.587320e+09,2761,Hyposmolality and/or hyponatremia
0,fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,6.700351e+09,F419,"Anxiety disorder, unspecified"
0,fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,6.613157e+09,2809,"Iron deficiency anemia, unspecified"


In [12]:
# Saving File
df.to_excel('./Output_File.xlsx', index=False)

# Note:
## Since multiple conditions are assigned to a single person, I have created new rows for each condition having same pid name